# Anomaly Detection Model

## Skripsi - Hagen Kwik

In [109]:
# library for mathematical functions and data preprocessing such as table, etc
import pandas as pd
import numpy as np

# Library for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Library for machine learning functions/algorithms
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import SGD, Nadam, Adam

# Use the legacy Adam optimizer
from mealpy import WOA, DE
from mealpy.utils.problem import FloatVar

#  Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

## Setup & EDA

### Import Libraries

### KDD99 Dataset

In [110]:
# data
df_kdd99_training = pd.read_csv('kdd99-training.csv')
df_kdd99_training.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [111]:
df_kdd99_testing = pd.read_csv('kdd99-testing.csv')
df_kdd99_testing.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
1,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.04,0.04,back.
2,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
3,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.02,0.02,back.
4,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.05,0.05,back.


In [112]:
df_kdd99_training.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [113]:
df_kdd99_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  num_compromised              494020 non-null  int64  
 13 

In [114]:
df_kdd99_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311028 entries, 0 to 311027
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     311028 non-null  int64  
 1   protocol_type                311028 non-null  object 
 2   service                      311028 non-null  object 
 3   flag                         311028 non-null  object 
 4   src_bytes                    311028 non-null  int64  
 5   dst_bytes                    311028 non-null  int64  
 6   land                         311028 non-null  int64  
 7   wrong_fragment               311028 non-null  int64  
 8   urgent                       311028 non-null  int64  
 9   hot                          311028 non-null  int64  
 10  num_failed_logins            311028 non-null  int64  
 11  logged_in                    311028 non-null  int64  
 12  num_compromised              311028 non-null  int64  
 13 

In [115]:
# Check which column needs fixing
for column in df_kdd99_testing.columns:
    unique_values = df_kdd99_testing[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[    0     9     2 ...  5049 10121  5033]

Unique values in 'protocol_type':
['tcp' 'icmp' 'udp']

Unique values in 'service':
['http' 'telnet' 'login' 'imap4' 'eco_i' 'private' 'smtp' 'whois' 'ssh'
 'ctf' 'ecr_i' 'domain' 'ftp' 'gopher' 'finger' 'rje' 'time' 'uucp' 'efs'
 'other' 'vmnet' 'klogin' 'netbios_ns' 'pop_2' 'netstat' 'remote_job'
 'shell' 'hostnames' 'ftp_data' 'http_443' 'netbios_ssn' 'iso_tsap'
 'csnet_ns' 'link' 'ldap' 'supdup' 'echo' 'pop_3' 'courier' 'name' 'nnsp'
 'exec' 'printer' 'systat' 'daytime' 'bgp' 'sql_net' 'auth' 'Z39_50' 'mtp'
 'kshell' 'uucp_path' 'netbios_dgm' 'sunrpc' 'nntp' 'discard' 'domain_u'
 'urp_i' 'IRC' 'ntp_u' 'urh_i' 'X11' 'tim_i' 'harvest']

Unique values in 'flag':
['SF' 'RSTR' 'RSTO' 'REJ' 'S0' 'SH' 'S1' 'S3' 'S2' 'OTH' 'RSTOS0']

Unique values in 'src_bytes':
[  54540   53168   54020 ...     520    1237 5133876]

Unique values in 'dst_bytes':
[     8314      7300      1460 ...      6944      4850 400291060]

Unique

In [116]:
# Check which column needs fixing
for column in df_kdd99_training.columns:
    unique_values = df_kdd99_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[   0    1   79 ... 2695 2751  120]

Unique values in 'protocol_type':
['tcp' 'udp' 'icmp']

Unique values in 'service':
['http' 'smtp' 'finger' 'domain_u' 'auth' 'telnet' 'ftp' 'eco_i' 'ntp_u'
 'ecr_i' 'other' 'private' 'pop_3' 'ftp_data' 'rje' 'time' 'mtp' 'link'
 'remote_job' 'gopher' 'ssh' 'name' 'whois' 'domain' 'login' 'imap4'
 'daytime' 'ctf' 'nntp' 'shell' 'IRC' 'nnsp' 'http_443' 'exec' 'printer'
 'efs' 'courier' 'uucp' 'klogin' 'kshell' 'echo' 'discard' 'systat'
 'supdup' 'iso_tsap' 'hostnames' 'csnet_ns' 'pop_2' 'sunrpc' 'uucp_path'
 'netbios_ns' 'netbios_ssn' 'netbios_dgm' 'sql_net' 'vmnet' 'bgp' 'Z39_50'
 'ldap' 'netstat' 'urh_i' 'X11' 'urp_i' 'pm_dump' 'tftp_u' 'tim_i' 'red_i']

Unique values in 'flag':
['SF' 'S1' 'REJ' 'S2' 'S0' 'S3' 'RSTO' 'RSTR' 'RSTOS0' 'OTH' 'SH']

Unique values in 'src_bytes':
[181 239 235 ... 500 484 475]

Unique values in 'dst_bytes':
[ 5450   486  1337 ... 34557 10592 13828]

Unique values in 'land':
[0 1]

Unique valu

In [117]:
missing_values_kdd99_testing = df_kdd99_testing.isnull().sum()
print(missing_values_kdd99_testing)

duplicates_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(duplicates_kdd99_testing)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [118]:
missing_values_kdd99_training = df_kdd99_training.isnull().sum()
print(missing_values_kdd99_training)

duplicates_kdd99_training = df_kdd99_training.duplicated().sum()
print(duplicates_kdd99_training)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

### UNSW-NB15 Dataset

In [119]:
# data
df_unsw_training = pd.read_csv('UNSW-training-set.csv')
df_unsw_testing = pd.read_csv('UNSW-testing-set.csv')
df_unsw_training.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [120]:
df_unsw_testing.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [121]:
# Check which column needs fixing
df_unsw_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [122]:
# Check which column needs fixing
df_unsw_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [123]:
# Check which column needs fixing
for column in df_unsw_training.columns:
    unique_values = df_unsw_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()
    

Unique values in 'id':
[     1      2      3 ... 175339 175340 175341]

Unique values in 'dur':
[0.121478 0.649902 1.623129 ... 3.71911  0.996503 1.557125]

Unique values in 'proto':
['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp'
 'wsn' 'pvp' 'wb-expak' 'mtp' 'pri-enc' 'sat-mon' 'cphb' 'sun-nd' 'iso-ip'
 'xtp' 'il' 'unas' 'mfe-nsp' '3pc' 'ipv6-route' 'idrp' 'bna' 'swipe'
 'kryptolan' 'cpnx' 'rsvp' 'wb-mon' 'vmtp' 'ib' 'dgp' 'eigrp' 'ax.25'
 'gmtp' 'pnni' 'sep' 'pgm' 'idpr-cmtp' 'zero' 'rvd' 'mobile' 'narp' 'fc'
 'pipe' 'ipcomp' 'ipv6-no' 'sat-expak' 'ipv6-opts' 'snp' 'ipcv'
 'br-sat-mon' 'ttp' 'tcf' 'nsfnet-igp' 'sprite-rpc' 'aes-sp3-d' 'sccopmce'
 'sctp' 'qnx' 'scps' 'etherip' 'aris' 'pim' 'compaq-peer' 'vrrp' 'iatp'
 'stp' 'l2tp' 'srp' 'sm' 'isis' 'smp' 'fire' 'ptp' 'crtp' 'sps'
 'merit-inp' 'idpr' 'skip' 'any' 'larp' 'ipip' 'micp' 'encap' 'ifmp'
 'tp++' 'a/n' 'ipv6' 'i-nlsp' 'ipx-n-ip' 'sdrp' 'tlsp' 'gre' 'mhrp' 'ddx'
 'ippc' 'visa' 'secure-vmtp' 'uti' 'vines

In [124]:
# Check for null values
missing_values_unsw_training = df_unsw_training.isnull().sum()
print(missing_values_unsw_training)

# Check for duplicate rows
duplicate_unsw_training = df_unsw_training.duplicated().sum()
print(duplicate_unsw_training)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

In [125]:
# Check for null values
missing_values_unsw_testing = df_unsw_testing.isnull().sum()
print(missing_values_unsw_testing)

# Check for duplicate rows
duplicate_unsw_testing = df_unsw_testing.duplicated().sum()
print(duplicate_unsw_testing)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

## Preprocessing

### Remove Duplicates

#### KDD99

In [126]:
print(f"Number of rows before removing duplicates: {len(df_kdd99_testing)}")

# Remove duplicates and overwrite the original DataFrame
df_kdd99_testing = df_kdd99_testing.drop_duplicates()

# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_testing)}")

# Check if any duplicates remain
remaining_duplicates_df_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates_df_kdd99_testing}")


Number of rows before removing duplicates: 311028
Number of rows after removing duplicates: 97325
Number of duplicate rows remaining: 0


#### UNSW

In [127]:
print(f"Number of rows before removing duplicates: {len(df_kdd99_training)}")

# Remove duplicates and overwrite the original DataFrame
df_kdd99_training = df_kdd99_training.drop_duplicates()

# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_training)}")

# Check if any duplicates remain
remaining_duplicates = df_kdd99_training.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates}")


Number of rows before removing duplicates: 494020
Number of rows after removing duplicates: 145585
Number of duplicate rows remaining: 0


### Label Encoding

#### KDD99

In [128]:
le = LabelEncoder() 
#training
df_kdd99_training['protocol_type'] = le.fit_transform(df_kdd99_training['protocol_type'])
df_kdd99_training['service'] = le.fit_transform(df_kdd99_training['service'])
df_kdd99_training['flag'] = le.fit_transform(df_kdd99_training['flag'])

#testing
df_kdd99_testing['protocol_type'] = le.fit_transform(df_kdd99_testing['protocol_type'])
df_kdd99_testing['service'] = le.fit_transform(df_kdd99_testing['service'])
df_kdd99_testing['flag'] = le.fit_transform(df_kdd99_testing['flag'])


#### UNSW

In [129]:
#training
df_unsw_training['state'] = le.fit_transform(df_unsw_training['state'])
df_unsw_training['service'] = le.fit_transform(df_unsw_training['service'])
df_unsw_training['proto'] = le.fit_transform(df_unsw_training['proto'])
#testing
df_unsw_testing['state'] = le.fit_transform(df_unsw_testing['state'])
df_unsw_testing['service'] = le.fit_transform(df_unsw_testing['service'])
df_unsw_testing['proto'] = le.fit_transform(df_unsw_testing['proto'])

### Remove Useless Columns

In [130]:
columns_to_drop_kdd = ['label']

# training
df_autoencoder_kdd99_train = df_kdd99_training[df_kdd99_training['label'] == 'normal'].drop(columns=columns_to_drop_kdd)

y = df_kdd99_training['label']
df_kdd99_training = df_kdd99_training.drop(columns=columns_to_drop_kdd)
X = df_kdd99_training

df_kdd99_train, df_kdd99_validation, df_kdd99_train_label, df_kdd99_validation_label = train_test_split(
    X, y, test_size=0.05, random_state=42, stratify=y
)

# testing
df_kdd99_testing_label = df_kdd99_testing['label']
df_kdd99_testing = df_kdd99_testing.drop(columns=columns_to_drop_kdd)

columns_to_drop_unsw = ['attack_cat', 'id', 'label']

# training
df_autoencoder_unsw_train = df_unsw_training[df_unsw_training['label'] == 0].drop(columns=columns_to_drop_unsw)

y = df_unsw_training['label']
df_unsw_training = df_unsw_training.drop(columns=columns_to_drop_unsw)
X = df_unsw_training

df_unsw_train, df_unsw_validation, df_unsw_train_label, df_unsw_validation_label = train_test_split(
    X, y, test_size=0.05, random_state=42, stratify=y
)

# testing
df_unsw_testing_label = df_unsw_testing['label']
df_unsw_testing = df_unsw_testing.drop(columns=columns_to_drop_unsw)

In [131]:
# KDD99 sampling
df_kdd99_train = df_kdd99_train.sample(frac=0.1, random_state=42)
df_kdd99_validation = df_kdd99_validation.sample(frac=0.1, random_state=42)
df_kdd99_testing = df_kdd99_testing.sample(frac=0.1, random_state=42)
df_autoencoder_kdd99_train = df_autoencoder_kdd99_train.sample(frac=0.1, random_state=42)

# UNSW sampling
df_unsw_train = df_unsw_train.sample(frac=0.1, random_state=42)
df_unsw_validation = df_unsw_validation.sample(frac=0.1, random_state=42)
df_unsw_testing = df_unsw_testing.sample(frac=0.1, random_state=42)
df_autoencoder_unsw_train = df_autoencoder_unsw_train.sample(frac=0.1, random_state=42)

# Update corresponding labels to match sampled data
df_kdd99_train_label = df_kdd99_train_label.loc[df_kdd99_train.index]
df_kdd99_validation_label = df_kdd99_validation_label.loc[df_kdd99_validation.index]
df_kdd99_testing_label = df_kdd99_testing_label.loc[df_kdd99_testing.index]

df_unsw_train_label = df_unsw_train_label.loc[df_unsw_train.index]
df_unsw_validation_label = df_unsw_validation_label.loc[df_unsw_validation.index]
df_unsw_testing_label = df_unsw_testing_label.loc[df_unsw_testing.index]

In [132]:
df_kdd99_validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 728 entries, 394742 to 381469
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     728 non-null    int64  
 1   protocol_type                728 non-null    int64  
 2   service                      728 non-null    int64  
 3   flag                         728 non-null    int64  
 4   src_bytes                    728 non-null    int64  
 5   dst_bytes                    728 non-null    int64  
 6   land                         728 non-null    int64  
 7   wrong_fragment               728 non-null    int64  
 8   urgent                       728 non-null    int64  
 9   hot                          728 non-null    int64  
 10  num_failed_logins            728 non-null    int64  
 11  logged_in                    728 non-null    int64  
 12  num_compromised              728 non-null    int64  
 13  root_shell       

In [133]:
df_autoencoder_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5600 entries, 95744 to 40350
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                5600 non-null   float64
 1   proto              5600 non-null   int64  
 2   service            5600 non-null   int64  
 3   state              5600 non-null   int64  
 4   spkts              5600 non-null   int64  
 5   dpkts              5600 non-null   int64  
 6   sbytes             5600 non-null   int64  
 7   dbytes             5600 non-null   int64  
 8   rate               5600 non-null   float64
 9   sttl               5600 non-null   int64  
 10  dttl               5600 non-null   int64  
 11  sload              5600 non-null   float64
 12  dload              5600 non-null   float64
 13  sloss              5600 non-null   int64  
 14  dloss              5600 non-null   int64  
 15  sinpkt             5600 non-null   float64
 16  dinpkt             5600 

In [134]:
df_autoencoder_kdd99_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8783 entries, 483253 to 456161
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     8783 non-null   int64  
 1   protocol_type                8783 non-null   int64  
 2   service                      8783 non-null   int64  
 3   flag                         8783 non-null   int64  
 4   src_bytes                    8783 non-null   int64  
 5   dst_bytes                    8783 non-null   int64  
 6   land                         8783 non-null   int64  
 7   wrong_fragment               8783 non-null   int64  
 8   urgent                       8783 non-null   int64  
 9   hot                          8783 non-null   int64  
 10  num_failed_logins            8783 non-null   int64  
 11  logged_in                    8783 non-null   int64  
 12  num_compromised              8783 non-null   int64  
 13  root_shell      

In [135]:
df_unsw_validation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 877 entries, 161082 to 12676
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                877 non-null    float64
 1   proto              877 non-null    int64  
 2   service            877 non-null    int64  
 3   state              877 non-null    int64  
 4   spkts              877 non-null    int64  
 5   dpkts              877 non-null    int64  
 6   sbytes             877 non-null    int64  
 7   dbytes             877 non-null    int64  
 8   rate               877 non-null    float64
 9   sttl               877 non-null    int64  
 10  dttl               877 non-null    int64  
 11  sload              877 non-null    float64
 12  dload              877 non-null    float64
 13  sloss              877 non-null    int64  
 14  dloss              877 non-null    int64  
 15  sinpkt             877 non-null    float64
 16  dinpkt             877 n

In [136]:
df_unsw_validation_label.info()

<class 'pandas.core.series.Series'>
Index: 877 entries, 161082 to 12676
Series name: label
Non-Null Count  Dtype
--------------  -----
877 non-null    int64
dtypes: int64(1)
memory usage: 13.7 KB


### Change labels to attack or not attack

In [137]:
df_kdd99_train_label = df_kdd99_train_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_validation_label = df_kdd99_validation_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_testing_label = df_kdd99_testing_label.apply(lambda x: 0 if x == 'normal.' else 1)

### Normalize Data

In [138]:
scaler = RobustScaler()

# kdd
df_kdd99_testing = pd.DataFrame(scaler.fit_transform(df_kdd99_testing), columns=df_kdd99_testing.columns)
df_autoencoder_kdd99_train = pd.DataFrame(scaler.fit_transform(df_autoencoder_kdd99_train),columns=df_autoencoder_kdd99_train.columns)
df_kdd99_train = pd.DataFrame(scaler.transform(df_kdd99_train),columns=df_kdd99_train.columns)
df_kdd99_validation = pd.DataFrame(scaler.transform(df_kdd99_validation),columns=df_kdd99_validation.columns)

# unsw
df_unsw_testing = pd.DataFrame(scaler.fit_transform(df_unsw_testing), columns=df_unsw_testing.columns)
df_autoencoder_unsw_train = pd.DataFrame(scaler.fit_transform(df_autoencoder_unsw_train),columns=df_autoencoder_unsw_train.columns)
df_unsw_train= pd.DataFrame(scaler.transform(df_unsw_train),columns=df_unsw_train.columns)
df_unsw_validation = pd.DataFrame(scaler.transform(df_unsw_validation),columns=df_unsw_validation.columns)


### Final Check for Dataset

#### KDD99

In [139]:
# kdd
df_kdd99_train_label.unique()


array([0, 1])

In [140]:
df_kdd99_testing_label.unique()


array([1, 0])

In [141]:
df_kdd99_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9732 entries, 0 to 9731
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     9732 non-null   float64
 1   protocol_type                9732 non-null   float64
 2   service                      9732 non-null   float64
 3   flag                         9732 non-null   float64
 4   src_bytes                    9732 non-null   float64
 5   dst_bytes                    9732 non-null   float64
 6   land                         9732 non-null   float64
 7   wrong_fragment               9732 non-null   float64
 8   urgent                       9732 non-null   float64
 9   hot                          9732 non-null   float64
 10  num_failed_logins            9732 non-null   float64
 11  logged_in                    9732 non-null   float64
 12  num_compromised              9732 non-null   float64
 13  root_shell        

In [142]:
df_autoencoder_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8783 entries, 0 to 8782
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     8783 non-null   float64
 1   protocol_type                8783 non-null   float64
 2   service                      8783 non-null   float64
 3   flag                         8783 non-null   float64
 4   src_bytes                    8783 non-null   float64
 5   dst_bytes                    8783 non-null   float64
 6   land                         8783 non-null   float64
 7   wrong_fragment               8783 non-null   float64
 8   urgent                       8783 non-null   float64
 9   hot                          8783 non-null   float64
 10  num_failed_logins            8783 non-null   float64
 11  logged_in                    8783 non-null   float64
 12  num_compromised              8783 non-null   float64
 13  root_shell        

In [143]:
df_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13830 entries, 0 to 13829
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     13830 non-null  float64
 1   protocol_type                13830 non-null  float64
 2   service                      13830 non-null  float64
 3   flag                         13830 non-null  float64
 4   src_bytes                    13830 non-null  float64
 5   dst_bytes                    13830 non-null  float64
 6   land                         13830 non-null  float64
 7   wrong_fragment               13830 non-null  float64
 8   urgent                       13830 non-null  float64
 9   hot                          13830 non-null  float64
 10  num_failed_logins            13830 non-null  float64
 11  logged_in                    13830 non-null  float64
 12  num_compromised              13830 non-null  float64
 13  root_shell      

In [144]:
df_kdd99_validation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     728 non-null    float64
 1   protocol_type                728 non-null    float64
 2   service                      728 non-null    float64
 3   flag                         728 non-null    float64
 4   src_bytes                    728 non-null    float64
 5   dst_bytes                    728 non-null    float64
 6   land                         728 non-null    float64
 7   wrong_fragment               728 non-null    float64
 8   urgent                       728 non-null    float64
 9   hot                          728 non-null    float64
 10  num_failed_logins            728 non-null    float64
 11  logged_in                    728 non-null    float64
 12  num_compromised              728 non-null    float64
 13  root_shell          

#### UNSW

In [145]:
df_unsw_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8233 entries, 0 to 8232
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                8233 non-null   float64
 1   proto              8233 non-null   float64
 2   service            8233 non-null   float64
 3   state              8233 non-null   float64
 4   spkts              8233 non-null   float64
 5   dpkts              8233 non-null   float64
 6   sbytes             8233 non-null   float64
 7   dbytes             8233 non-null   float64
 8   rate               8233 non-null   float64
 9   sttl               8233 non-null   float64
 10  dttl               8233 non-null   float64
 11  sload              8233 non-null   float64
 12  dload              8233 non-null   float64
 13  sloss              8233 non-null   float64
 14  dloss              8233 non-null   float64
 15  sinpkt             8233 non-null   float64
 16  dinpkt             8233 

In [146]:
df_autoencoder_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5600 entries, 0 to 5599
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                5600 non-null   float64
 1   proto              5600 non-null   float64
 2   service            5600 non-null   float64
 3   state              5600 non-null   float64
 4   spkts              5600 non-null   float64
 5   dpkts              5600 non-null   float64
 6   sbytes             5600 non-null   float64
 7   dbytes             5600 non-null   float64
 8   rate               5600 non-null   float64
 9   sttl               5600 non-null   float64
 10  dttl               5600 non-null   float64
 11  sload              5600 non-null   float64
 12  dload              5600 non-null   float64
 13  sloss              5600 non-null   float64
 14  dloss              5600 non-null   float64
 15  sinpkt             5600 non-null   float64
 16  dinpkt             5600 

In [147]:
df_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16657 entries, 0 to 16656
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                16657 non-null  float64
 1   proto              16657 non-null  float64
 2   service            16657 non-null  float64
 3   state              16657 non-null  float64
 4   spkts              16657 non-null  float64
 5   dpkts              16657 non-null  float64
 6   sbytes             16657 non-null  float64
 7   dbytes             16657 non-null  float64
 8   rate               16657 non-null  float64
 9   sttl               16657 non-null  float64
 10  dttl               16657 non-null  float64
 11  sload              16657 non-null  float64
 12  dload              16657 non-null  float64
 13  sloss              16657 non-null  float64
 14  dloss              16657 non-null  float64
 15  sinpkt             16657 non-null  float64
 16  dinpkt             166

In [148]:
df_unsw_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 877 entries, 0 to 876
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                877 non-null    float64
 1   proto              877 non-null    float64
 2   service            877 non-null    float64
 3   state              877 non-null    float64
 4   spkts              877 non-null    float64
 5   dpkts              877 non-null    float64
 6   sbytes             877 non-null    float64
 7   dbytes             877 non-null    float64
 8   rate               877 non-null    float64
 9   sttl               877 non-null    float64
 10  dttl               877 non-null    float64
 11  sload              877 non-null    float64
 12  dload              877 non-null    float64
 13  sloss              877 non-null    float64
 14  dloss              877 non-null    float64
 15  sinpkt             877 non-null    float64
 16  dinpkt             877 non

In [149]:
df_unsw_train_label.unique()

array([1, 0])

In [150]:
df_kdd99_testing_label.unique()

array([1, 0])

## **Model Generation with Optimizer + Evaluation**

### +++++++++++++++++++++++++++++++++++++++++++++++ **Autoencoder** +++++++++++++++++++++++++++++++++++++++++++++++

In [151]:
def create_autoencoder(
    input_dim,latent_dim,neurons_per_layer,activation_function, output_activation, loss_function, optimizer, learning_rate, dropout_rate):
    # Input Layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    x = Dense(neurons_per_layer, activation=activation_function)(input_layer)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    encoded = Dense(latent_dim, activation=activation_function)(x)  # Latent space
    
    # Decoder
    x = Dense(neurons_per_layer, activation=activation_function)(encoded)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    decoded = Dense(input_dim, activation=output_activation)(x)  # Output layer
    
    # Autoencoder Model
    autoencoder = Model(input_layer, decoded)

    # Optimizer Selection
    if optimizer == "Adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "SGD":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "Nadam":
        opt = Nadam(learning_rate=learning_rate)
    
    # Compile Model
    autoencoder.compile(optimizer=opt, loss=loss_function)
    
    return autoencoder

In [152]:
def objective_function(solution, X_train, X_val, y_val, input_dim, dataset_name):
    (latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs,
     activation_idx, output_activation_idx, loss_idx, optimizer_idx,) = solution

    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    activation_function = activation_functions[int(activation_idx)]
    output_activation = output_activations[int(output_activation_idx)]
    loss_function = loss_functions[int(loss_idx)]
    optimizer = optimizers[int(optimizer_idx)]

    # Create Autoencoder
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
    )

    # Train Autoencoder
    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Compute reconstruction error on training data for threshold calculation
    reconstructed_train = autoencoder.predict(X_train)
    mse_train = np.mean(np.square(X_train - reconstructed_train), axis=1)
    
    # Compute reconstruction error on validation data (fitness value)
    reconstructed_val = autoencoder.predict(X_val)
    mse_val = np.mean(np.square(X_val - reconstructed_val), axis=1)
    
    # Set threshold percentile based on dataset
    threshold_percentile = 40 if dataset_name == "unsw" else 60
    threshold = np.percentile(mse_train, threshold_percentile)
    
    # Calculate validation accuracy (fitness value to maximize)
    y_val_pred = (mse_val > threshold).astype(int)
    accuracy = accuracy_score(y_val, y_val_pred)
    
    return -accuracy  # Negative because we're minimizing the objective

In [153]:
def optimize_autoencoder(X_train, X_val, y_val, input_dim, optimizer_type, dataset_name, epochs=20, pop_size=10):
    # Define the search space
    problem_dict = {
        "bounds": FloatVar(
            lb=[
                16,    # latent_dim (16-64)
                16,    # neurons_per_layer (16-128)
                0.001, # learning_rate (0.001-0.01)
                0.1,   # dropout_rate (0.1-0.5)
                256,    # batch_size (16-64)
                20,    # epochs (20-100)
                0,     # activation_function index (0: relu, 1: tanh, 2: sigmoid)
                0,     # output_activation index (0: sigmoid, 1: tanh, 2: linear, 3: softmax, 4: relu)
                0,     # loss_function index (0: mse, 1: mae, 2: binary_crossentropy, 3: categorical_crossentropy)
                0,     # optimizer index (0: Adam, 1: SGD, 2: Nadam)
            ],
            ub=[
                64,128,0.01,0.5,256,100,2,4,3,2,],
            name="delta",
        ),
        "minmax": "min",  # Minimize MSE
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val, input_dim, dataset_name),
    }
    
    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=10, n_jobs=-1)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size, miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1, n_jobs=-1)
    else:
        raise ValueError("Optimizer not supported. Choose 'WOA' or 'DE'.")
    
    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution

In [154]:
def evaluate_model(autoencoder, X_train, X_test, y_test, dataset_name):
    reconstructed_train = autoencoder.predict(X_train)
    mse_train = np.mean(np.square(X_train - reconstructed_train), axis=1)
    
    # Calculate reconstruction error for test data
    reconstructed_test = autoencoder.predict(X_test)
    mse_test = np.mean(np.square(X_test - reconstructed_test), axis=1)
    
    # Define threshold based on dataset (40 for UNSW, 60 for KDD)
    threshold_percentile = 40 if dataset_name == "unsw" else 60
    threshold = np.percentile(mse_train, threshold_percentile)
    
    # Predict anomalies
    y_pred = (mse_test > threshold).astype(int)
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, mse_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return accuracy, precision, recall, f1, auc, conf_matrix, threshold

In [155]:
def main_pipeline(dataset_name, optimizer_type):
    # Load Dataset
    if dataset_name == "kdd99":
        X_train = df_kdd99_train
        X_val = df_kdd99_validation
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
        y_val = df_kdd99_validation_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
        y_val = df_unsw_validation_label
    else:
        raise ValueError("Dataset not supported. Choose 'kdd99' or 'unsw'.")

    input_dim = X_train.shape[1]

    # Optimize Autoencoder (now passing dataset_name directly)
    best_params = optimize_autoencoder(
        X_train, 
        X_val,
        y_val,
        input_dim, 
        optimizer_type,
        dataset_name
    )

    # Rest of the function remains the same...
    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    (
        latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs, activation_idx, output_activation_idx,
        loss_idx, optimizer_idx,) = best_params

    activation_function = activation_functions[int(activation_idx)]
    output_activation = output_activations[int(output_activation_idx)]
    loss_function = loss_functions[int(loss_idx)]
    optimizer = optimizers[int(optimizer_idx)]

    # Train Final Model
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
    )

    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Evaluate (pass dataset_name to evaluate_model)
    accuracy, precision, recall, f1, auc, conf_matrix, threshold = evaluate_model(
        autoencoder, X_train, X_test, y_test, dataset_name)
    
    print(f"\n=== For Dataset {dataset_name}, {optimizer_type} optimizer ===")
    print(f"Threshold Percentile Used: {40 if dataset_name == 'unsw' else 60}")
    print(f"Calculated Threshold Value: {threshold:.4f}")

    print("\n=== Best Parameters ===")
    print(f"Latent Dim: {int(latent_dim)}")
    print(f"Neurons/Layer: {int(neurons_per_layer)}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Dropout Rate: {dropout_rate}")
    print(f"Batch Size: {int(batch_size)}")
    print(f"Epochs: {int(epochs)}")
    print(f"Activation: {activation_function}")
    print(f"Output Activation: {output_activation}")
    print(f"Loss Function: {loss_function}")
    print(f"Optimizer: {optimizer}")

    print("\n=== Evaluation Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

#### ************Autoencoder + Binary Differential Evolution Algorithm************

In [156]:
main_pipeline('kdd99', 'DE')

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 02:42:59 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433

2025/03/27 02:46:22 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: -0.8447802197802198, Global best: -0.8447802197802198, Runtime: 99.43810 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:24:48 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: -0.9395604395604396, Global best: -0.9395604395604396, Runtime: 162.60767 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:27:38 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: -0.9395604395604396, Global best: -0.9395604395604396, Runtime: 170.09932 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step

=== For Dataset kdd99, DE optimizer ===
Threshold Percentile Used: 60
Calculated Threshold Value: 2918446107055.1709

=== Best Parameters ===
Latent Dim: 38
Neurons/Layer: 100
Learning Rate: 0.007919439264665502
Dropout Rate: 0.2984549616826757
Batch Size: 256
Epochs: 77
Activation: relu
Output Activation: linear
Loss Function: categorical_crossentropy
Optimizer: Adam

=== Evaluation Metrics ===
Accuracy: 0.5563
Precision: 0.0593
Recall: 0.0039
F1 Score: 0.0074
AUC: 0.8254
Confusion Matrix:
[[5398  254]
 [4064   16]]


In [157]:
main_pipeline('unsw', 'DE')

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:28:12 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/

2025/03/27 03:32:02 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: -0.7297605473204105, Global best: -0.7297605473204105, Runtime: 112.00616 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:01:32 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: -0.7559863169897377, Global best: -0.7559863169897377, Runtime: 124.00140 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:04:03 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: -0.7559863169897377, Global best: -0.7559863169897377, Runtime: 150.89215 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step
258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step

=== For Dataset unsw, DE optimizer ===
Threshold Percentile Used: 40
Calculated Threshold Value: 1461.4598

=== Best Parameters ===
Latent Dim: 17
Neurons/Layer: 96
Learning Rate: 0.004610828298159693
Dropout Rate: 0.45382521469407544
Batch Size: 256
Epochs: 75
Activation: relu
Output Activation: sigmoid
Loss Function: binary_crossentropy
Optimizer: Adam

=== Evaluation Metrics ===
Accuracy: 0.3886
Precision: 0.1084
Recall: 0.0145
F1 Score: 0.0256
AUC: 0.3407
Confusion Matrix:
[[3133  543]
 [4491   66]]


#### ************Autoencoder + Whale Optimization Algorithm************

In [158]:
main_pipeline('kdd99', 'WOA')

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:04:28 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/

2025/03/27 04:09:08 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: -0.8090659340659341, Global best: -0.8090659340659341, Runtime: 162.73575 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 05:08:25 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: -0.8777472527472527, Global best: -0.9217032967032966, Runtime: 200.24437 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 05:11:35 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: -0.9381868131868132, Global best: -0.9381868131868132, Runtime: 189.48011 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step

=== For Dataset kdd99, WOA optimizer ===
Threshold Percentile Used: 60
Calculated Threshold Value: 93.7520

=== Best Parameters ===
Latent Dim: 48
Neurons/Layer: 128
Learning Rate: 0.007759272157046315
Dropout Rate: 0.5
Batch Size: 256
Epochs: 100
Activation: sigmoid
Output Activation: relu
Loss Function: categorical_crossentropy
Optimizer: Nadam

=== Evaluation Metrics ===
Accuracy: 0.5059
Precision: 0.4187
Recall: 0.4603
F1 Score: 0.4385
AUC: 0.5253
Confusion Matrix:
[[3045 2607]
 [2202 1878]]


In [159]:
main_pipeline('unsw', 'WOA')

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 05:12:15 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 791us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/

2025/03/27 05:16:44 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: -0.7559863169897377, Global best: -0.7559863169897377, Runtime: 156.50139 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 761us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 06:30:37 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: -0.8038768529076397, Global best: -0.8038768529076397, Runtime: 275.17139 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 792us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 774us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 06:35:18 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: -0.8038768529076397, Global best: -0.8038768529076397, Runtime: 281.42568 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step

=== For Dataset unsw, WOA optimizer ===
Threshold Percentile Used: 40
Calculated Threshold Value: 98673008126408671232.0000

=== Best Parameters ===
Latent Dim: 64
Neurons/Layer: 128
Learning Rate: 0.01
Dropout Rate: 0.5
Batch Size: 256
Epochs: 100
Activation: relu
Output Activation: relu
Loss Function: categorical_crossentropy
Optimizer: SGD

=== Evaluation Metrics ===
Accuracy: 0.2864
Precision: 0.3545
Recall: 0.3522
F1 Score: 0.3533
AUC: 0.2393
Confusion Matrix:
[[ 753 2923]
 [2952 1605]]
